In [ ]:
import cv2
import numpy as np
import time
import random
from tkinter import Tk, filedialog
from PIL import Image, ImageTk
from collections import Counter

# --- Configuration ---
OPTIMAL_PH_LOWER = 6.0
OPTIMAL_PH_UPPER = 7.5
YELLOW_DISCOLORATION_THRESHOLD = 0.03
BRIGHT_SPOT_THRESHOLD = 220
MIN_GRAIN_AREA = 50
K_NEIGHBORS = 3  # Number of neighbors to consider for KNN

# --- Simulated Training Data for pH Prediction (Yellow % vs. pH) ---
training_data_ph = [
    (0.01, 6.5),  # Low yellow, near optimal pH
    (0.02, 6.8),
    (0.005, 7.0),
    (0.05, 5.5),  # Higher yellow, lower pH
    (0.06, 5.2),
    (0.04, 5.8),
    (0.015, 7.2), # Low yellow, slightly high pH
    (0.025, 7.8),
]

def load_image_from_user():
    root = Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select Crop Image",
        filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.jfif")]
    )
    return file_path

def analyze_crop_image_elaborate(image_path):
    analysis_results = {"discoloration": {"yellow": False, "brown": False, "yellow_percentage": 0.0},
                        "spots": False,
                        "wilting": False,
                        "overall_visual_health_score": 1.0}
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not load image at {image_path}")
            return analysis_results, None, None

        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        total_pixels = img.shape[0] * img.shape[1]

        lower_yellow = np.array([20, 100, 100])
        upper_yellow = np.array([40, 255, 255])
        mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
        yellow_pixels = cv2.countNonZero(mask_yellow)
        analysis_results["discoloration"]["yellow"] = (yellow_pixels / total_pixels) > YELLOW_DISCOLORATION_THRESHOLD
        analysis_results["discoloration"]["yellow_percentage"] = yellow_pixels / total_pixels

        lower_brown = np.array([10, 50, 50])
        upper_brown = np.array([20, 255, 200])
        mask_brown = cv2.inRange(hsv, lower_brown, upper_brown)
        brown_pixels = cv2.countNonZero(mask_brown)
        analysis_results["discoloration"]["brown"] = (brown_pixels / total_pixels) > YELLOW_DISCOLORATION_THRESHOLD

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(blurred)
        analysis_results["spots"] = max_val > BRIGHT_SPOT_THRESHOLD

        health_score = 1.0
        if analysis_results["discoloration"]["yellow"]:
            health_score -= 0.3
        if analysis_results["discoloration"]["brown"]:
            health_score -= 0.4
        if analysis_results["spots"]:
            health_score -= 0.2
        analysis_results["overall_visual_health_score"] = np.clip(health_score, 0.0, 1.0)

        return analysis_results, mask_yellow, mask_brown

    except Exception as e:
        print(f"Image analysis error: {e}")
        return analysis_results, None, None

def predict_ph_knn(yellow_percentage, training_data, k):
    """Predicts pH using a KNN-like approach based on yellow percentage."""
    distances = []
    for yellow_train, ph_train in training_data:
        distance = abs(yellow_percentage - yellow_train)
        distances.append((distance, ph_train))
    distances.sort(key=lambda x: x[0])
    neighbors = distances[:k]
    neighbor_phs = [ph for dist, ph in neighbors]
    # Simple average of the neighbors' pH values
    predicted_ph = sum(neighbor_phs) / len(neighbor_phs)
    return round(predicted_ph, 2)

def assess_crop_quality_with_ai_ph(predicted_ph, visual_analysis):
    quality_report = {"overall_quality": "Uncertain", "details": [], "ai_analysis": []}
    ph_score = 0.0

    # --- AI-Driven pH Analysis ---
    quality_report["ai_analysis"].append(f"AI (KNN-like) predicted pH: {predicted_ph}")

    if OPTIMAL_PH_LOWER <= predicted_ph <= OPTIMAL_PH_UPPER:
        quality_report["details"].append(f"Predicted pH is optimal ({predicted_ph}) - Suggests good quality.")
        ph_score = 1.0
        if visual_analysis["overall_visual_health_score"] < 0.8:
            quality_report["ai_analysis"].append("Despite optimal predicted pH, visual indicators show some issues. Consider other factors.")
    elif predicted_ph < OPTIMAL_PH_LOWER:
        quality_report["details"].append(f"Predicted pH is low ({predicted_ph}) - Indicates potential issues.")
        ph_score = 0.5
        if visual_analysis["discoloration"]["yellow"]:
            quality_report["ai_analysis"].append("Low predicted pH aligns with yellow discoloration, possibly due to nutrient deficiencies.")
    else: # predicted_ph > OPTIMAL_PH_UPPER
        quality_report["details"].append(f"Predicted pH is high ({predicted_ph}) - Indicates potential issues.")
        ph_score = 0.5
        if visual_analysis["discoloration"]["brown"] or visual_analysis["spots"]:
            quality_report["ai_analysis"].append("High predicted pH correlates with brown discoloration or spots, possibly due to nutrient imbalances or toxicity.")

    # --- Visual Analysis Summary ---
    if visual_analysis["discoloration"]["yellow"]:
        quality_report["ai_analysis"].append(f"Yellow discoloration detected ({visual_analysis['discoloration']['yellow_percentage']:.2f}).")
    if visual_analysis["discoloration"]["brown"]:
        quality_report["ai_analysis"].append("Brown discoloration detected.")
    if visual_analysis["spots"]:
        quality_report["ai_analysis"].append("Bright spots detected.")

    # --- Overall Quality (primarily based on predicted pH and visual context) ---
    if ph_score > 0.8 and visual_analysis["overall_visual_health_score"] > 0.7:
        quality_report["overall_quality"] = "High Quality"
    elif ph_score >= 0.5:
        quality_report["overall_quality"] = "Potentially Good Quality"
    else:
        quality_report["overall_quality"] = "Low Quality"

    quality_report["overall_quality_score"] = (ph_score + visual_analysis["overall_visual_health_score"]) / 2.0
    return quality_report

def display_analyzed_image(image_path, yellow_mask, brown_mask):
    img = cv2.imread(image_path)
    if img is not None:
        yellow_highlight = cv2.bitwise_and(img, img, mask=yellow_mask)
        brown_highlight = cv2.bitwise_and(img, img, mask=brown_mask)
        combined_highlight = cv2.addWeighted(yellow_highlight, 0.7, brown_highlight, 0.7, 0)

        cv2.imshow("Original Image", img)
        cv2.imshow("Yellow Discoloration Highlight", yellow_highlight)
        cv2.imshow("Brown Discoloration Highlight", brown_highlight)
        cv2.imshow("Combined Discoloration", combined_highlight)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

if __name__ == "__main__":
    print("Crop Quality Assessment System with AI for pH Analysis")

    print("\nSelect the crop image to analyze:")
    image_path = load_image_from_user()
    if image_path:
        visual_data, mask_yellow, mask_brown = analyze_crop_image_elaborate(image_path)
        print(f"Visual Analysis: {visual_data}")

        # AI predicts pH based on yellow percentage
        predicted_ph = predict_ph_knn(visual_data["discoloration"]["yellow_percentage"], training_data_ph, K_NEIGHBORS)
        print(f"AI Predicted pH (KNN-like): {predicted_ph}")

        quality_report = assess_crop_quality_with_ai_ph(predicted_ph, visual_data)
        print("Quality Assessment with AI pH Analysis:")
        for detail in quality_report["details"]:
            print(f"- {detail}")
        if quality_report["ai_analysis"]:
            print("\nAI Analysis:")
            for analysis in quality_report["ai_analysis"]:
                print(f"- {analysis}")
        print(f"Overall Quality: {quality_report['overall_quality']} (Score: {quality_report['overall_quality_score']:.2f})")
        print("-" * 30)

        if mask_yellow is not None and mask_brown is not None:
            display_analyzed_image(image_path, mask_yellow, mask_brown)
    else:
        print("No image selected for analysis.")
